In [74]:
#Imports
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

sns.set(style="darkgrid")

In [37]:
liste_merk = ["ID", "NACHNAME", "VORNAME","GESCHLECHT", "PATREI_KURZ","GEBURTSDATUM","BERUF", "GEBURTSLAND"]

"MANDATSART", "WP",""

In [52]:
#Import

import xml.etree.ElementTree as ET


def read_xml(path):
   xml = ET.parse(path).getroot()
   return xml

def search_for_name(tree, attr):
   names = []
   for child in tree:
       if child.tag == attr:
           names += [child.text]
       else:
           names += search_for_name(child, attr)
   return names

#Test
if __name__=="__main__":
   xml = read_xml("/Users/Laszlo Kuehl/Desktop/Project_12/mdb-stammdaten-data/MDB_STAMMDATEN.XML")
   names = search_for_name(xml,"GESCHLECHT")
   

print(len(names))

4073


In [79]:
#Erste Ansätze für ein DataFrame
column_list = []
for attr in liste_merk:
    if __name__=="__main__":
        xml = read_xml("/Users/Laszlo Kuehl/Desktop/Project_12/mdb-stammdaten-data/MDB_STAMMDATEN.XML")
        names = search_for_name(xml, attr)
        column_list.append(names)

In [191]:
liste_merk = ["ID", "NACHNAME", "VORNAME","GESCHLECHT", "PARTEI_KURZ","GEBURTSDATUM","BERUF", "GEBURTSLAND"]
liste_merk_2 = ["ID","GESCHLECHT", "PARTEI_KURZ","GEBURTSDATUM","BERUF", "GEBURTSLAND"]

TypeError: list indices must be integers or slices, not str

In [81]:
print(len(column_list))

for i in column_list:
    print(len(i))

8
4073
4336
4336
4073
4073
4073
4073
4073


In [120]:
df_first.dtypes

ID              object
GESCHLECHT      object
PARTEI_KURZ     object
GEBURTSDATUM    object
BERUF           object
GEBURTSLAND     object
dtype: object

In [143]:
df_first = pd.DataFrame(np.column_stack([column_list[0],column_list[3],column_list[4],column_list[5],column_list[6],column_list[7]]), columns = liste_merk_2)

df_first.loc[df_first["GEBURTSLAND"].isnull(), "GEBURTSLAND"] = "Deutschland"


print(df_first.head())

   

         ID GESCHLECHT PARTEI_KURZ GEBURTSDATUM  \
0  11000001   männlich         CDU         1930   
1  11000002   männlich         FDP         1909   
2  11000003   weiblich         CDU         1913   
3  11000004   weiblich         CDU         1933   
4  11000005   männlich         CDU         1950   

                                               BERUF  GEBURTSLAND  
0  Rechtsanwalt, Wirtschaftsprüfer, Universitätsp...  Deutschland  
1                             Rechtsanwalt und Notar  Deutschland  
2                                    Hilfsreferentin  Jugoslawien  
3                                             Ärztin  Deutschland  
4                      Mathematiker, Geschäftsführer  Deutschland  


In [144]:
if __name__=="__main__":
   xml = read_xml("/Users/Laszlo Kuehl/Desktop/Project_12/mdb-stammdaten-data/MDB_STAMMDATEN.XML")
   names = search_for_name(xml, "WP")

    
print(len(names))

11600


In [147]:
def read_xml(path):
 xml = ET.parse(path).getroot()
 return xml

def search_for_nameRoots(root):
 names = []
 for child in root:
     if child.tag == "NAMEN":
         names += [child]
     else:
         names += search_for_nameRoots(child)
 return names

xml = read_xml(path = "/Users/Laszlo Kuehl/Desktop/Project_12/mdb-stammdaten-data/MDB_STAMMDATEN.XML")

nameRoots = search_for_nameRoots(xml)

firstNameElements = []
for element in nameRoots:
   firstNameElements += [element[0].find('VORNAME').text]

lastNameElements = []
for element in nameRoots:
   lastNameElements += [element[0].find('NACHNAME').text]



In [150]:
df_first["NACHNAME"] = lastNameElements
df_first["VORNAME"] = firstNameElements

In [152]:
df_first.head()

,ID,GESCHLECHT,PARTEI_KURZ,GEBURTSDATUM,BERUF,GEBURTSLAND,NACHNAME,VORNAME
0,11000001,männlich,CDU,1930,"Rechtsanwalt, Wirtschaftsprüfer, Universitätsp...",Deutschland,Abelein,Manfred
1,11000002,männlich,FDP,1909,Rechtsanwalt und Notar,Deutschland,Achenbach,Ernst
2,11000003,weiblich,CDU,1913,Hilfsreferentin,Jugoslawien,Ackermann,Annemarie
3,11000004,weiblich,CDU,1933,Ärztin,Deutschland,Ackermann,Else
4,11000005,männlich,CDU,1950,"Mathematiker, Geschäftsführer",Deutschland,Adam,Ulrich


In [202]:
def read_xml(path):
    xml = ET.parse(path).getroot()
    return xml

def search_for_nameRoots1(root):
    names = []
    for child in root:
        if child.tag == "WAHLPERIODEN":
            names += [search_for_nameRoots2(child)]
        else:
            names += search_for_nameRoots1(child)
    return names


def search_for_nameRoots2(root2):
    names2 = []
    for child2 in root2:
        if child2.tag == "WP":
            names2 += [child2]
        elif child2.tag == "ID":
            break
        else:
            names2 += search_for_nameRoots2(child2)
    return names2


xml = read_xml(path = "/Users/Laszlo Kuehl/Desktop/Project_12/mdb-stammdaten-data/MDB_STAMMDATEN.XML")

WP = search_for_nameRoots1(xml)
    
    
def extracting_WP(list1):
    WPelements = []
    for i in list1:
        sub_list = []
        for a in i:
            sub_list += [a.text]
        WPelements += [sub_list]
    return WPelements
        
        
    

w_perioden = extracting_WP(WP)

print(len(w_perioden))



4073
